# Acquire

**Goal: Your goal is to predict the values of single unit properties using the obervations from 2017.**

**import**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sys
from sklearn.model_selection import train_test_split

sys.path.append("./util_")
import acquire_
import prepare_

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

**get data**

In [2]:
# sql query
query = """
SELECT *
FROM properties_2017 AS prop
INNER JOIN predictions_2017 USING(parcelid)
LEFT OUTER JOIN airconditioningtype USING(airconditioningtypeid)
LEFT OUTER JOIN architecturalstyletype USING(architecturalstyletypeid)
LEFT OUTER JOIN buildingclasstype USING(buildingclasstypeid)
LEFT OUTER JOIN heatingorsystemtype USING(heatingorsystemtypeid)
LEFT OUTER JOIN propertylandusetype USING(propertylandusetypeid)
LEFT OUTER JOIN storytype USING(storytypeid)
LEFT OUTER JOIN typeconstructiontype USING(typeconstructiontypeid)
WHERE latitude IS NOT NULL AND longitude IS NOT NULL;
"""

In [3]:
# get data from codeup database
zillow, q = acquire_.get_codeup_sql_data_(db_name="zillow", query=query,fileName="zillow_single_family")

## Understand data

In [4]:
zillow.shape

(77580, 69)

In [5]:
zillow.columns

Index(['typeconstructiontypeid', 'storytypeid', 'propertylandusetypeid',
       'heatingorsystemtypeid', 'buildingclasstypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'parcelid', 'id',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17',

In [6]:
zillow.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77580 entries, 0 to 77579
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        223 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77580 non-null  float64
 3   heatingorsystemtypeid         49572 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      207 non-null    float64
 6   airconditioningtypeid         25007 non-null  float64
 7   parcelid                      77580 non-null  int64  
 8   id                            77580 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77580 non-null  float64
 11  bedroomcnt                    77580 non-null  float64
 12  buildingqualitytypeid         49810 non-null  float64
 13  c

In [7]:
zillow.describe()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id,logerror
count,223.00,50.00,"77,580.00","49,572.00",15.00,207.00,"25,007.00","77,580.00","77,580.00",50.00,...,172.00,"77,465.00","77,579.00","77,580.00","77,578.00","77,575.00","2,900.00","77,333.00","77,580.00","77,580.00"
mean,6.04,7.00,261.82,3.92,3.93,7.39,1.81,"13,008,279.03","1,495,403.96",679.72,...,1.00,"189,279.64","490,147.61","2,016.00","301,150.00","5,995.93",14.09,"60,496,672,300,497.99","38,806.72",0.02
std,0.56,0.00,5.14,3.59,0.26,2.73,2.97,"3,519,375.97","860,970.00",689.70,...,0.00,"230,409.55","653,794.17",0.00,"492,721.93","7,628.82",2.18,"1,533,329,446,287.88","22,403.76",0.17
min,4.00,7.00,31.00,1.00,3.00,2.00,1.00,"10,711,855.00",349.00,38.00,...,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00",0.00,-4.66
25%,6.00,7.00,261.00,2.00,4.00,7.00,1.00,"11,538,203.50","752,143.00",273.00,...,1.00,"84,171.00","206,899.00","2,016.00","85,293.25","2,712.65",14.00,"60,373,109,004,005.00","19,404.75",-0.02
50%,6.00,7.00,261.00,2.00,4.00,7.00,1.00,"12,530,558.50","1,498,256.00",515.00,...,1.00,"136,402.00","358,878.00","2,016.00","203,181.00","4,448.23",15.00,"60,376,032,001,021.00","38,804.50",0.01
75%,6.00,7.00,266.00,7.00,4.00,7.00,1.00,"14,211,351.25","2,240,950.50",796.50,...,1.00,"218,734.00","569,000.00","2,016.00","366,739.75","6,926.89",15.00,"60,590,423,251,007.00","58,208.25",0.04
max,13.00,7.00,275.00,24.00,4.00,21.00,13.00,"167,689,317.00","2,982,274.00","3,560.00",...,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00","77,613.00",5.26


In [8]:
# count of numeric columns and object columns
numerics = len(zillow.select_dtypes("number").columns)
objects = len(zillow.select_dtypes("object").columns)

print("Numeric col count:", numerics)
print("object col count:", objects)

Numeric col count: 58
object col count: 11


Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [9]:
# create a dataframe from all the null counts
zillow_null_status = pd.DataFrame(zillow.isna().sum(axis=0))
zillow_null_status = zillow_null_status.rename(columns={0:"num_rows_missing"})

# add percentage of th missing values to the new data frame
zillow_null_status["pct_rows_missing"] = zillow_null_status.num_rows_missing / zillow.shape[0]
zillow_null_status.head()

,num_rows_missing,pct_rows_missing
typeconstructiontypeid,77357,1.00
storytypeid,77530,1.00
propertylandusetypeid,0,0.00
heatingorsystemtypeid,28008,0.36
buildingclasstypeid,77565,1.00


**What I see:**

- I have 2152863 rows and 7 columns
- 7 of the 29 columns are numric while 0 of them are string object colums
- I have unsure null values in my data
- I also see the descriptive statistics of my data

# Prepare

In [10]:
zillow.T

,0,1,2,3,4,5,6,7,8,9,...,77570,77571,77572,77573,77574,77575,77576,77577,77578,77579
typeconstructiontypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storytypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
propertylandusetypeid,261.00,261.00,261.00,261.00,266.00,266.00,261.00,261.00,261.00,266.00,...,266.00,266.00,261.00,261.00,266.00,261.00,261.00,261.00,261.00,261.00
heatingorsystemtypeid,NaN,NaN,NaN,2.00,2.00,NaN,2.00,NaN,2.00,2.00,...,2.00,2.00,2.00,2.00,2.00,2.00,NaN,2.00,2.00,7.00
buildingclasstypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
buildingclassdesc,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
heatingorsystemdesc,None,None,None,Central,Central,None,Central,None,Central,Central,...,Central,Central,Central,Central,Central,Central,None,Central,Central,Floor/Wall
propertylandusedesc,Single Family Residential,Single Family Residential,Single Family Residential,Single Family Residential,Condominium,Condominium,Single Family Residential,Single Family Residential,Single Family Residential,Condominium,...,Condominium,Condominium,Single Family Residential,Single Family Residential,Condominium,Single Family Residential,Single Family Residential,Single Family Residential,Single Family Residential,Single Family Residential
storydesc,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


**Handle missing values**

Remove rows that are 75% emply and columns taht are 50% empty

In [11]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75, col_drop = []):
    # round the required columns times the length of the data frame to 0
    # remove every column that is 50% empty
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)

    # round the required rows times the length of the data frame to 0
    # remove every row that is 75% empty
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    
    # drop more specific columns
    df = df.drop(columns=col_drop)
    return df

In [12]:
# columns to drop
col_drop= ["propertylandusetypeid",
          "heatingorsystemtypeid",
           "buildingqualitytypeid",
           "calculatedfinishedsquarefeet",
           "calculatedbathnbr",
           ""
          "id",
           "heatingorsystemdesc",
           "propertyzoningdesc",
           "transactiondate"
          ]

zillow = handle_missing_values(df = zillow, col_drop = col_drop)
zillow.head()

,parcelid,bathroomcnt,bedroomcnt,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,propertylandusedesc
0,14297519,3.50,4.00,"3,100.00","6,059.00",3.00,"33,634,931.00","-117,869,207.00","4,506.00",122,...,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0.03,2017-01-01,Single Family Residential
1,17052889,1.00,2.00,"1,465.00","6,111.00",1.00,"34,449,266.00","-119,281,531.00","12,647.00",1110,...,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",0.06,2017-01-01,Single Family Residential
2,14186244,2.00,3.00,"1,243.00","6,059.00",2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,...,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",0.01,2017-01-01,Single Family Residential
3,12177905,3.00,4.00,"2,376.00","6,037.00",3.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,...,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",-0.10,2017-01-01,Single Family Residential
4,10887214,3.00,3.00,"1,312.00","6,037.00",3.00,"34,185,120.00","-118,414,640.00","278,581.00",010C,...,"1,964.00","73,681.00","119,407.00","2,016.00","45,726.00","1,533.89","60,371,236,012,000.00",0.01,2017-01-01,Condominium


**Rename columns**

In [13]:
# rename dataframe columns
zillow = zillow.rename(columns={
    "bedroomcnt":"bedrooms",
    "bathroomcnt":"bathrooms",
    "finishedsquarefeet12":"sqr_feet",
    "taxvaluedollarcnt":"tax_value",
    "yearbuilt":"year_built",
    "taxamount":"tax_amount",
    "fips":"county"
})


In [14]:
zillow.county.value_counts()

6,037.00    50661
6,059.00    20531
6,111.00     6232
Name: county, dtype: int64

**Remove duplicates**

In [15]:
zilow = zillow.drop_duplicates(subset="parcelid", keep="first")

In [16]:
zillow.county.value_counts()

6,037.00    50661
6,059.00    20531
6,111.00     6232
Name: county, dtype: int64

**Fill null**

In [17]:
# Find all the columns that contains null values
cols = zillow.columns

for ele in cols:
    null_count = zillow[ele].isna().sum()
    
    # numeric columns
    if (null_count != 0) and (zillow[ele].dtypes != "O"):
        # fill nulls with median
        zillow[ele] = zillow[ele].fillna(zillow[ele].median())
        
#     elif null_count != 0:
#         # remove all nulls for object data types
#         zillow[ele] = zillow[ele].fillna("This is null")

In [18]:
# zillow = zillow[zillow.heatingorsystemdesc != "This is null"]
# # zillow = zillow[zillow.propertyzoningdesc != "This is null"]


**convert data type**

In [19]:
numeric_cols_to_covert = zillow.select_dtypes("float").drop(columns=["rawcensustractandblock","tax_amount","logerror", 
                                                           "tax_value","landtaxvaluedollarcnt","structuretaxvaluedollarcnt"]).columns
# convert data type from float to int
for i in numeric_cols_to_covert:
    zillow[i] = zillow[i].astype(int)


**Remove outliers**

This is done using box plot in the explore phase. only looking at the training data.

The IQR is major for determining outliers.

In [20]:
# add a new column to the data frame that indicates the outliers in the numeric columns.
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [21]:
def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df

In [22]:
zillow_outliers = add_upper_outlier_columns(zillow.copy(), k=1.5)
zillow_outliers

,parcelid,bathrooms,bedrooms,sqr_feet,county,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,roomcnt_outliers,unitcnt_outliers,year_built_outliers,structuretaxvaluedollarcnt_outliers,tax_value_outliers,assessmentyear_outliers,landtaxvaluedollarcnt_outliers,tax_amount_outliers,censustractandblock_outliers,logerror_outliers
0,14297519,3,4,3100,6059,3,33634931,-117869207,4506,122,...,0.00,0.00,0,"66,283.50",0.00,0.00,0.00,0.00,0.00,0.00
1,17052889,1,2,1465,6111,1,34449266,-119281531,12647,1110,...,5.00,0.00,0,0.00,0.00,0.00,0.00,0.00,"193,616,944,991.12",0.00
2,14186244,2,3,1243,6059,2,33886168,-117823170,8432,122,...,6.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,12177905,3,4,2376,6037,3,34245180,-118240722,13038,0101,...,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,10887214,3,3,1312,6037,3,34185120,-118414640,278581,010C,...,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77575,11000655,2,2,1286,6037,2,34245368,-118282383,47405,0100,...,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
77576,17239384,2,4,1612,6111,2,34300140,-118706327,12105,1111,...,7.00,0.00,0,0.00,0.00,0.00,0.00,0.00,"193,690,944,001.12",0.00
77577,12773139,1,3,1032,6037,1,34040895,-118038169,5074,0100,...,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
77578,12826780,2,3,1762,6037,2,33937685,-117996709,6347,0100,...,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [23]:
# see what the outliers in our data look like:
outlier_cols = [col for col in zillow_outliers if col.endswith('_outliers')]
for col in outlier_cols:
    print('~~~\n' + col)
    data = zillow_outliers[col][zillow_outliers[col] > 0]
    print(data.describe())

~~~
parcelid_outliers
count                  28.00
mean          142,799,832.16
std            21,178,645.24
min            61,423,483.62
25%           148,250,263.12
50%           149,419,720.12
75%           149,420,309.12
max           149,471,525.62
Name: parcelid_outliers, dtype: float64
~~~
bathrooms_outliers
count               1,992.00
mean                    1.11
std                     1.03
min                     0.50
25%                     0.50
50%                     0.50
75%                     1.50
max                    13.50
Name: bathrooms_outliers, dtype: float64
~~~
bedrooms_outliers
count                 383.00
mean                    1.64
std                     1.17
min                     1.00
25%                     1.00
50%                     1.00
75%                     2.00
max                     9.00
Name: bedrooms_outliers, dtype: float64
~~~
sqr_feet_outliers
count               4,468.00
mean                1,042.25
std                 1,314.27
min    

In [24]:
# remove outliers
zillow = zillow[zillow.bedrooms < 5]
zillow = zillow[zillow.bathrooms < 5]
zillow = zillow[(zillow.year_built >= 1910) & (zillow.year_built <= 2017)]
zillow = zillow[(zillow.sqr_feet > 500) & (zillow.sqr_feet <= 3800)]
zillow = zillow[zillow.tax_amount <= 20000]
zillow = zillow[zillow.tax_value <= 1_000_000]

**feature engineeing**

In [25]:
# Rename the unique values in fips to county names
zillow.county = zillow.county.astype(str).str.replace("6037","Los Angeles").str.replace("6059","Orange").str.replace("6111","Sam Juan")

## Split

In [26]:
# split the data into training, validation and testing sets
train, validate, test = prepare_.split_data_(df=zillow,
                    test_size=0.2, 
                     validate_size=0.2, 
                     random_state=95)
(train.shape, validate.shape, test.shape)

((39405, 26), (13135, 26), (13136, 26))

**Save split**

In [27]:
prepare_.save_split_data(encoded_df=zillow, train=train, validate=validate, test=test)

'Four data sets saved as .csv'